# Preamble

## Library


In [ ]:
# load packages
import requests
import time
import json
import re
from bs4 import BeautifulSoup
from pprint import pprint

## PAT Token


In [ ]:
# define pats and api tokens safely via gitignored file 
from secrets import gh_api_token
from secrets import ai_api_token

## Global Parameters


In [ ]:
# Define repository and category IDs
repository_id = 'R_kgDOKYhvWw'
category_id = 'DIC_kwDOKYhvW84CZobi'

# Define most recent message ID
recent_id = 8687
# recent_id = 8677

# define id where to stop
# stop_id = 8664
stop_id = 8685

# Define number of messages we want to retrieve (for testing)
n_msg = (recent_id - stop_id)+1

## Functions


In [ ]:
############################### fetch_details ########################################

# define fetching function based on msg number
def fetch_details(msg_number):
    # Initialize an empty dictionary to hold the details
    details = {}

    # Add the message number to the dictionary
    details['id'] = msg_number
    
    # Generate the full URL of the detailed page
    url = f"https://www.mail-archive.com/nmusers@globomaxnm.com/msg{msg_number}.html"
    
    # Fetch the HTML content from the URL
    response = requests.get(url)
    page_content = response.text
    
    # Initialize a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(page_content, 'html.parser')

    # Additional fields for thread details
    thread_section = soup.select_one('div.tSliceList ul.icons')
    
    # Check if the list is empty
    details['is_standalone'] = len(thread_section.find_all('li', recursive=False)) == 0 if thread_section else True
                                   
    # Extract message IDs within the thread
    details['thread_message_ids'] = []
    if not details['is_standalone']:
        thread_msgs = thread_section.select('a[href^="msg"]')
        details['thread_message_ids'] = [msg['href'].replace('msg', '').replace('.html', '') for msg in thread_msgs]

    # append id to thread_message_ids
    details['thread_message_ids'].append(msg_number)

    # sort message ids in upscending order
    details['thread_message_ids'] = sorted(details['thread_message_ids'], key=int)
    
    # Extract the date
    date_tag = soup.select_one('span.date a')
    if date_tag:
        details['date'] = date_tag.text.strip()

    # Extract the subject
    subject_tag = soup.select_one('span.subject span[itemprop="name"]')
    if subject_tag:
        details['subject'] = subject_tag.text.strip()

    # Extract the author
    author_tag = soup.select_one('span.sender span[itemprop="name"]')
    if author_tag:
        details['author'] = author_tag.text.strip()
    
    # Extract the message text
    message_tag = soup.select_one('div.msgBody')
    if message_tag:
        details['message'] = message_tag.text.strip()

    return details

############################### create_discussion ####################################

# define function to create discussion in GitHub repo
def create_discussion(api_token, repository_id, category_id, date, author, title, body):
    # define url
    url = 'https://api.github.com/graphql'

    # define additional information (pre)
    add_inf_pre = f'**Date:** {date}\n**Author:** {author}\n\n'

    # defione additional information (post)
    add_inf_post = f'\n\n---\n\n*Please note: this discussion was automatically created via web scraping of the nmusers mail archive. If you have any questions, please contact the original author of this message. If you are the original author and want your message deleted, you can contact the maintainer at any time.*'

    # replace '"' with '\"' in body
    body = body.replace('"', '\\"')

    # add additional information to body
    body = add_inf_pre + body + add_inf_post

    # define query
    query = f'mutation{{createDiscussion(input: {{repositoryId: "{repository_id}", categoryId: "{category_id}", body: "{body}", title: "{title}"}}) {{discussion {{id}}}}}}'

    # define json paylod
    json_payload  = { 'query' : query}

    # define headers
    headers = {'Authorization': 'token %s' % api_token}

    # post request and return json
    r = requests.post(url=url, json=json_payload, headers=headers)

    # return response text and store as dict
    return json.loads(r.text)

############################ add_comment_to_discussion ################################

# Function to add a comment to an existing discussion in GitHub
def add_comment_to_discussion(api_token, discussion_id, body, date, author, silent):
    # Define GraphQL URL
    url = 'https://api.github.com/graphql'

    # define additional information (pre)
    add_inf_pre = f'**Date:** {date}\n**Author:** {author}\n\n'

    # defione additional information (post)
    add_inf_post = f'\n\n---\n\n*Please note: this discussion was automatically created via web scraping of the nmusers mail archive. If you have any questions, please contact the original author of this message. If you are the original author and want your message deleted, you can contact the maintainer at any time.*'
    
    # Replace '"' with '\"' in body
    body = body.replace('"', '\\"')

    # add additional information to body
    body = add_inf_pre + body + add_inf_post
    
    # Define GraphQL query for adding a comment
    query = f'''mutation{{
        addDiscussionComment(input: {{ body: "{body}", discussionId: "{discussion_id}"}}) {{
            comment {{
                id
            }}
        }}
    }}'''
    
    # Define JSON payload
    json_payload  = {'query': query}
    
    # Define headers
    headers = {'Authorization': f'token {api_token}'}
    
    # Make POST request
    r = requests.post(url=url, json=json_payload, headers=headers)
    
    # Print response if not silent
    if not silent:
        print(r.text + "\n")


################################ extract_threads ######################################

# function to extract the threads from a list of messages
def extract_threads(messages_dict):
    thread_dict = {}
    thread_id = 1  # Starting thread ID

    # To keep track of messages that are already part of a thread
    seen_messages = set() 

    # Loop through each message
    for msg_id, msg in messages_dict.items():
        
        # Skip this message if it's already part of a thread
        if msg_id in seen_messages:
            continue
        
        # Extract the thread message IDs
        thread_message_ids = msg['thread_message_ids']
        
        # add threads
        thread_dict[thread_id] = thread_message_ids
        thread_id += 1
        
        # Mark all messages in this thread as seen
        seen_messages.update(thread_message_ids)
            
    return thread_dict


############################### fetch_missing_messages ###############################

# Function to fetch missing messages in thread_dict and add them to msg dictionary
def fetch_missing_messages(thread_dict, msg):
    for thread_id, msg_list in thread_dict.items():
        for msg_id in msg_list:
            if msg_id not in msg:
                # Fetch missing message details
                fetched_msg = fetch_details(msg_id)
                
                # Add the fetched message to msg dictionary
                msg[msg_id] = fetched_msg


############################### delete_discussion ###################################

# Function to delete a discussion by its ID
def delete_discussion(api_token, discussion_id):
    # Define GraphQL URL
    url = 'https://api.github.com/graphql'
    
    # Define GraphQL query for deleting a discussion
    query = f'''mutation{{
        deleteDiscussion(input: {{ id: "{discussion_id}" }}) {{
            clientMutationId
        }}
    }}'''
    
    # Define JSON payload
    json_payload  = {'query': query}
    
    # Define headers
    headers = {'Authorization': f'token {api_token}'}
    
    # Make POST request
    r = requests.post(url=url, json=json_payload, headers=headers)
    
    # Check for errors
    if r.status_code == 200:
        print(f'Successfully deleted discussion with ID {discussion_id}.')
    else:
        print(f'Failed to delete discussion with ID {discussion_id}. Error: {r.text}')


############################### list_all_discussions #################################

# Function to list all discussions by repository ID
def list_all_discussions(api_token, repository_id):
    # Define GraphQL URL
    url = 'https://api.github.com/graphql'

    # Define GraphQL query for fetching discussions
    query = f'''
    query {{
      node(id: "{repository_id}") {{
        ... on Repository {{
          discussions(first: 100) {{
            nodes {{
              id
              title
            }}
          }}
        }}
      }}
    }}
    '''
    
    # Define JSON payload
    json_payload  = {'query': query}

    # Define headers
    headers = {'Authorization': f'token {api_token}'}

    # Make POST request
    r = requests.post(url=url, json=json_payload, headers=headers)

    # Parse the JSON response
    discussions_data = json.loads(r.text)

    # Extract discussion ids and titles
    discussions = [(d['id'], d['title']) for d in discussions_data['data']['node']['discussions']['nodes']]

    return discussions


# Scraping

## Fetching List of Messages


In [ ]:
# Define vector of message IDs
msg_ids = list(range(recent_id, recent_id - n_msg, -1))

# Convert numerics to strings and paste 0 in front
msg_ids = ["0" + str(x) for x in msg_ids]

# show
msg_ids

# Initialize an empty dictionary to hold the msg information
msg = {}

# Loop through each msg id, extract the details and append
for id in msg_ids:

    # Fetch the details
    single_msg = fetch_details(id)

    # Append to msg
    msg[id] = single_msg


# Extract threads
thread_dict = extract_threads(msg)

# Fetch any missing messages in the threads
fetch_missing_messages(thread_dict, msg)

# Pretty print the result
pprint(thread_dict, indent=4)

## Loop


In [ ]:
# Loop through each element in the dictionary
for thread_id, msg_list in thread_dict.items():
    # define id
    id = msg_list[0]

    # retrieve message
    cur_msg = msg[id]

    # create discussion for id and store output
    dis_out = create_discussion(
        api_token=gh_api_token,
        title=cur_msg['subject'],
        body=cur_msg['message'],
        date=cur_msg['date'],
        author=cur_msg['author'],
        repository_id = repository_id,
        category_id = category_id
    )

    # output to console
    print(f'Created discussion {dis_out["data"]["createDiscussion"]["discussion"]["id"]} for message {id}.')

    # store freshly created discussion id
    dis_id = dis_out['data']['createDiscussion']['discussion']['id']

    # check if thread has more than one message
    if len(msg_list) > 1:
        # loop through all messages in thread
        for msg_id in msg_list[1:]:
            # retrieve message
            cur_msg = msg[msg_id]

            # add comment to discussion
            add_comment_to_discussion(
                api_token=gh_api_token,
                discussion_id=dis_id,
                body=cur_msg['message'],
                date=cur_msg['date'],
                author=cur_msg['author'],
                silent=False
            )
            
            # output to console
            print(f'Added comment to discussion {dis_id} for message {msg_id}.')

## Delete all discussions (if needed)


In [ ]:
# # Fetch all discussions for the given repository ID
# all_discussions = list_all_discussions(api_token=gh_api_token, repository_id=repository_id)

# # Delete each discussion
# for dis_id, title in all_discussions:
#     print(f"Deleting discussion with ID: {dis_id}, Title: {title}")
#     delete_discussion(api_token=gh_api_token, discussion_id=dis_id)